In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# Implement this!
def dense_layer(input_layer,
                num_outputs,
                activation_fn=tf.nn.relu):
    raise NotImplementedError


def dense_network(inputs,
                  hidden_sizes=[256,128],
                  num_labels=10,
                  activation_fn=tf.nn.relu):
    layers = [inputs]
    for i, hidden_size in enumerate(hidden_sizes + [num_labels]):
        with tf.variable_scope("layer{}".format(i)):
            next_layer = dense_layer(layers[-1],
                                     hidden_size,
                                     activation_fn=activation_fn)
            layers.append(next_layer)
        
    return layers[-1]

In [ ]:
tf.reset_default_graph()

# input placeholder
input_images = tf.placeholder(tf.float32, [None, 784]) # 28**2 = 784

# true labels
true_labels = tf.placeholder(tf.float32, [None, 10])

# network output (we call it logits because these are the unnormalized inputs to the softmax)
logits = dense_network(input_images)
    
# loss function 
# tf.nn.softmax_cross_entropy_with_logits does the same thing as before, with more numerical stability
cross_entropy = \
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                            labels=true_labels,
                                            name="cross_entropy")

# optimizer (Adam is the current de-facto standard. An overview over backpropagation algorithms can be found here:
# http://sebastianruder.com/optimizing-gradient-descent/index.html)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

train_step = optimizer.minimize(cross_entropy)

# evaluation
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(true_labels,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# initialization
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    
    summary_writer = tf.summary.FileWriter("./graph", graph=sess.graph)
    
    for i in range(20000):
        # get a minibatch of size 100
        batch_xs, batch_ys = mnist.train.next_batch(100)
        
        # run the training step
        feed_dict = {input_images: batch_xs,
                     true_labels: batch_ys}
        sess.run(train_step, feed_dict=feed_dict)
        
        if i%1000==0 and i>0:
            print(i)
    
    test_accuracy = sess.run(accuracy, feed_dict={input_images: mnist.test.images,
                                                  true_labels: mnist.test.labels})
    
print(test_accuracy)